## Extract tree cover data from three different tree cover datasets at the label points created in the bastin tree cover potential.

In [1]:
# import earth engine and intialise high volume end-point
import ee
# ee.Authenticate() # you may need to authenticate
ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

In [14]:
# import os

# import csv
# import geedim
# import geemap

from geeml.utils import eeprint, getCountry, createGrid
from geeml.extract import extractor

from tqdm.auto import tqdm

In [3]:
%load_ext watermark
%watermark
%watermark --iversions

Last updated: 2024-04-04T16:02:08.458126+02:00

Python implementation: CPython
Python version       : 3.9.13
IPython version      : 8.4.0

Compiler    : MSC v.1929 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 165 Stepping 2, GenuineIntel
CPU cores   : 12
Architecture: 64bit

ee: 0.2



In [4]:
# Import data
points = ee.FeatureCollection("projects/ee-geethensingh/assets/bastin_points")


In [5]:
# potapov
gf = ee.ImageCollection("projects/sat-io/open-datasets/GLAD/GEDI_V27")
gbf = ee.ImageCollection("projects/sat-io/open-datasets/GLAD/GEDI_V25_Boreal")
comb = gf.merge(gbf).mosaic().rename('potapov')

# Reiner
tree_cover = ee.Image("projects/sat-io/open-datasets/PS_AFRICA_TREECOVER_2019_100m_V10").rename('reiner')

# Sexton
GFCC30TC = ee.ImageCollection("projects/sat-io/open-datasets/GFCC30TC")
# GFFC2000 = GFCC30TC.filterDate('2000-01-01','2001-12-31').mosaic()
GFFC2015 = GFCC30TC.filterDate('2015-01-01','2015-12-31').mosaic().rename('sexton_2015')

covariates = comb.addBands([tree_cover, GFFC2015])
eeprint(covariates.bandNames())

In [6]:
import geemap
Map = geemap.Map()
Map.addLayer(covariates)
Map.addLayer(points)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [9]:
# Define global bounds
# globalBounds = ee.Geometry.Polygon([-180, 90, 0, 90, 180, 90, 180, -90, 10, -90, -180, -90], None, False)

# Print the global bounds
Map.addLayer(covariates.select([1]).geometry().bounds())


In [15]:
continents = ee.FeatureCollection("projects/ee-geethensingh/assets/World_Continents")
continent_names = continents.aggregate_array('CONTINENT').getInfo()

for continent_name in tqdm(continent_names):
    geom = continents.filter(ee.Filter.eq('CONTINENT', continent_name)).geometry()
    print(f"Extracting data for {continent_name}")
    # Download directory
    dd = r'C:\Users\coach\myfiles\scratch'
    # Initialise extractor
    gedi = extractor(covariates = covariates, target = points, aoi = geom, scale = 70, dd=dd, spcvGridSize= 30000)
    # Extract data in batches of 30 000 points
    gedi.extractPoints(gridSize = 100000, batchSize = 30000, filename = f"bastin_covariates_{continent_name}.csv")

  0%|          | 0/8 [00:00<?, ?it/s]

Extracting data for Africa


bastin_covariates_Africa.csv: |          | [  0.0%] in 00:00 (eta:     ?)

Extracting data for Asia


bastin_covariates_Asia.csv: |          | [  0.0%] in 00:00 (eta:     ?)

...

bastin_covariates_South America.csv: |          | [  0.0%] in 00:00 (eta:     ?)

Extracting data for Antarctica


bastin_covariates_Antarctica.csv: |          | [  0.0%] in 00:00 (eta:     ?)

Extracting data for Europe


bastin_covariates_Europe.csv: |          | [  0.0%] in 00:00 (eta:     ?)